# TensorFlow 2.0 Bert models on GLUE

*from* <a href=https://github.com/huggingface/transformers/blob/master/examples/run_tf_glue.py>*github.transformers*</a><br>
Fine-tuning the library TensorFlow 2.0 Bert model for sequence classification on the MRPC task of the GLUE benchmark: <a href=https://gluebenchmark.com/>General Language Understanding Evaluation</a>.

In [5]:
import os
import tensorflow as tf
import tensorflow_datasets
import torch
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification, glue_convert_examples_to_features, AlbertForSequenceClassification
tf.compat.v2.test.is_gpu_available()

True

In [6]:
# script parameters
BATCH_SIZE = 32
EVAL_BATCH_SIZE = BATCH_SIZE * 2
USE_XLA = False
USE_AMP = False

In [7]:
tf.config.optimizer.set_jit(USE_XLA)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": USE_AMP})

In [8]:
# Load tokenizer and model from pretrained model/vocabulary
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v1')
model = TFAlbertForSequenceClassification.from_pretrained('albert-base-v1')

ResourceExhaustedError: OOM when allocating tensor with shape[30000,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]

In [6]:
# Load dataset via TensorFlow Datasets
data, info = tensorflow_datasets.load('glue/mrpc', with_info=True)
train_examples = info.splits['train'].num_examples
valid_examples = info.splits['validation'].num_examples

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Reusing dataset glue (/root/tensorflow_datasets/glue/mrpc/0.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from /root/tensorflow_datasets/glue/mrpc/0.0.2


In [7]:
# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, 128, 'mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, 128, 'mrpc')
train_dataset = train_dataset.shuffle(128).batch(BATCH_SIZE).repeat(-1)
valid_dataset = valid_dataset.batch(EVAL_BATCH_SIZE)

In [8]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule 
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
if USE_AMP:
    # loss scaling is currently required when using mixed precision
    opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt, 'dynamic')
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [9]:
#try put stuff on cuda
model.to('cuda')
train_dataset.to('cuda')
valid_dataset.to('cuda')
train_dataset.to('cuda')
valid_dataset.to('cuda')

AttributeError: 'TFAlbertForSequenceClassification' object has no attribute 'to'

In [10]:
# Train and evaluate using tf.keras.Model.fit()
train_steps = train_examples//BATCH_SIZE
valid_steps = valid_examples//EVAL_BATCH_SIZE

In [14]:
import datetime
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
!kill 145

/bin/sh: 1: kill: No such process



In [12]:
root_logdir = "../.logs"

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_logdir)

In [13]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3,
                                                    restore_best_weights=True)

In [16]:
history = model.fit(train_dataset, epochs=100, 
                    steps_per_epoch=train_steps,
                    validation_data=valid_dataset, 
                    validation_steps=valid_steps,
                    callbacks=[tensorboard_callback, early_stopping_cb])

Train for 114 steps, validate for 6 steps
Epoch 1/100
114/114 [==============================] - 97s 849ms/step - loss: 0.6494 - accuracy: 0.6656 - val_loss: 0.6210 - val_accuracy: 0.6875
Epoch 2/100
114/114 [==============================] - 82s 718ms/step - loss: 0.6388 - accuracy: 0.6760 - val_loss: 0.6232 - val_accuracy: 0.6875
Epoch 3/100
114/114 [==============================] - 82s 720ms/step - loss: 0.6432 - accuracy: 0.6727 - val_loss: 0.6272 - val_accuracy: 0.6875
Epoch 4/100
114/114 [==============================] - 82s 720ms/step - loss: 0.6340 - accuracy: 0.6697 - val_loss: 0.6207 - val_accuracy: 0.6875
Epoch 5/100
114/114 [==============================] - 82s 721ms/step - loss: 0.6236 - accuracy: 0.6727 - val_loss: 0.5971 - val_accuracy: 0.6849
Epoch 6/100
 16/114 [===>..........................] - ETA: 1:06 - loss: 0.6038 - accuracy: 0.6816WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy


KeyboardInterrupt: 

In [15]:
%tensorboard --logdir=../.logs --host=0.0.0.0 #--bind_all 

In [15]:
# Save TF2 model
os.makedirs('../models/', exist_ok=True)
model.save_pretrained('../models/')

In [16]:
# Load the TensorFlow model in PyTorch for inspection
pytorch_model = BertForSequenceClassification.from_pretrained('../models/', from_tf=True)

In [17]:
# Quickly test a few predictions - MRPC is a paraphrasing task, let's see if our model learned the task
sentence_0 = 'This research was consistent with his findings.'
sentence_1 = 'His findings were compatible with this research.'
sentence_2 = 'His findings were not applicable to this paper.'
inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt')
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=True, return_tensors='pt')

del inputs_1["special_tokens_mask"]  # <---- add this
del inputs_2["special_tokens_mask"]  # <---- add this

pred_1 = pytorch_model(**inputs_1)[0].argmax().item()
pred_2 = pytorch_model(**inputs_2)[0].argmax().item()
print('sentence_1 is', 'a paraphrase' if pred_1 else 'not a paraphrase', 'of sentence_0')
print('sentence_2 is', 'a paraphrase' if pred_2 else 'not a paraphrase', 'of sentence_0')

sentence_1 is a paraphrase of sentence_0
sentence_2 is a paraphrase of sentence_0
